In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}
// Stops auto-scrolling so entire output is visible: see https://stackoverflow.com/a/41646403

In [ ]:
# Default parameter values. They will be overwritten by papermill notebook parameters.
# This cell must carry the tag "parameters" in its metadata.
from pathlib import Path
import pickle
import codecs

innereye_path = Path.cwd().parent.parent.parent
train_metrics_csv = ""
val_metrics_csv = innereye_path / 'Tests' / 'ML' / 'reports' / 'val_metrics_classification.csv'
test_metrics_csv = innereye_path / 'Tests' / 'ML' / 'reports' / 'test_metrics_classification.csv'
number_best_and_worst_performing = 20
config= ""

In [ ]:
import sys
print(f"Adding to path: {innereye_path}")
if str(innereye_path) not in sys.path:
    sys.path.append(str(innereye_path))

%matplotlib inline
import matplotlib.pyplot as plt

config = pickle.loads(codecs.decode(config.encode(), "base64"))

from InnerEye.ML.reports.notebook_report import print_header
from InnerEye.ML.reports.classification_report import plot_pr_and_roc_curves_from_csv, \
print_k_best_and_worst_performing, print_metrics, plot_k_best_and_worst_performing, get_labels_and_predictions

import warnings
warnings.filterwarnings("ignore")
plt.rcParams['figure.figsize'] = (20, 10)

#convert params to Path
train_metrics_csv = Path(train_metrics_csv)
val_metrics_csv = Path(val_metrics_csv)
test_metrics_csv = Path(test_metrics_csv)

# Train Metrics

In [ ]:
if train_metrics_csv.is_file():
    for label in config.class_names:
        print_header(f"Class {label}", level=3)
        train_metrics = get_labels_and_predictions(train_metrics_csv, label)
        print_metrics(val_metrics=train_metrics, test_metrics=train_metrics, exclusive=False)

# Validation Metrics

In [ ]:
if val_metrics_csv.is_file():
    for label in config.class_names:
        print_header(f"Class {label}", level=3)
        val_metrics = get_labels_and_predictions(val_metrics_csv, label)
        print_metrics(val_metrics=val_metrics, test_metrics=val_metrics, exclusive=False)

# Test Metrics

In [ ]:
if val_metrics_csv.is_file() and test_metrics_csv.is_file():
    for label in config.class_names:
        print_header(f"Class {label}", level=3)
        val_metrics = get_labels_and_predictions(val_metrics_csv, label)
        test_metrics = get_labels_and_predictions(test_metrics_csv, label)
        print_metrics(val_metrics=val_metrics, test_metrics=test_metrics, exclusive=False)

# AUC and PR curves
## Test Set

In [ ]:
if test_metrics_csv.is_file():
    for hue in config.class_names:
        print_header(f"Class {hue}", level=3)
        test_metrics = get_labels_and_predictions(test_metrics_csv, hue)
        plot_pr_and_roc_curves_from_csv(test_metrics)

## Validation set

In [ ]:
if val_metrics_csv.is_file():
    for hue in config.class_names:
        print_header(f"Class {hue}", level=3)
        val_metrics = get_labels_and_predictions(val_metrics_csv, hue)
        plot_pr_and_roc_curves_from_csv(val_metrics)

## Training set

In [ ]:
if train_metrics_csv.is_file():
    for hue in config.class_names:
        print_header(f"Class {hue}", level=3)
        train_metrics = get_labels_and_predictions(train_metrics_csv, hue)
        plot_pr_and_roc_curves_from_csv(train_metrics)

# Best and worst samples by ID

In [ ]:
if val_metrics_csv.is_file() and test_metrics_csv.is_file():
    for hue in config.class_names:
        print_header(f"Class {hue}", level=3)
        print_k_best_and_worst_performing(val_metrics_csv=val_metrics_csv, test_metrics_csv=test_metrics_csv,
                                      k=number_best_and_worst_performing,
                                      hue=hue)

# Plot best and worst sample images

In [ ]:
if val_metrics_csv.is_file() and test_metrics_csv.is_file():
    for hue in config.class_names:
        print_header(f"Class {hue}", level=3)
        plot_k_best_and_worst_performing(val_metrics_csv=val_metrics_csv, test_metrics_csv=test_metrics_csv,
                                     k=number_best_and_worst_performing, hue=hue, config=config)